In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk import bigrams, pos_tag




In [ ]:
askindia_labelled=pd.read_csv('/kaggle/input/final-project/AskIndia_labelled_final_v2.csv')

In [ ]:
india_labelled=pd.read_csv('/kaggle/input/final-project/India_labelled.csv')

In [ ]:
india_comments_total=pd.concat([askindia_labelled,india_labelled], ignore_index=True)

In [ ]:
india_comments_total

In [ ]:
india_comments_total=india_comments_total.dropna()

In [ ]:
india_comments_total.to_csv('india_comments_total.csv',index=False)

In [ ]:
india_comments_total=pd.read_csv('/kaggle/input/final-project/india_comments_total.csv')

In [ ]:
labels = india_comments_total['results']

# Map labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Convert encoded labels to one-hot encoding
onehot_labels = to_categorical(encoded_labels)

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
india_comments_total['comments']=[remove_stopwords(x) for x in india_comments_total['comments']]

In [ ]:
def extract_bigrams(text):
    try:
        tokens = word_tokenize(text)
        bigram_tokens = list(bigrams(tokens))
        return [' '.join(bigram) for bigram in bigram_tokens]
    except Exception as e:
        return []

train_bigrams = [extract_bigrams(comment) for comment in india_comments_total['comments']]

# Extract POS tags
def extract_pos_tags(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    return [tag[1] for tag in pos_tags]

train_pos_tags = [extract_pos_tags(comment) for comment in india_comments_total['comments']]

In [ ]:
train_pos_tags = [extract_pos_tags(comment) for comment in train_data]

# Combine original comments, bigrams, and POS tags
train_combined_features = [' '.join([comment] + bigram + pos_tags)
                           for comment, bigram, pos_tags in zip(train_data, train_bigrams, train_pos_tags)]


In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(train_combined_features, train_labels, test_size=0.2, random_state=69)

In [ ]:
all_words = [word for comment in train_data for word in word_tokenize(comment)]
vocab_size = len(Counter(all_words))

In [ ]:
vocab_size

In [ ]:
max_words = vocab_size
max_len = 500
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(train_data)
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)
train_padded = pad_sequences(train_sequences, maxlen=max_len)
test_padded = pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
EMBEDDING_FILE = '/kaggle/input/final-project/glove.twitter.27B.50d.txt'
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_words, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_words: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(max_words, output_dim=embed_size, weights=[embedding_matrix], input_length=max_len, trainable=False))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, return_sequences=True))  # Use return_sequences=True for stacking LSTM layers
# model.add(LSTM(100, kernel_regularizer=l2(0.01)))
# model.add(Dense(128, activation='relu'))  
# # model.add(Dense(64, activation='relu'))   
# model.add(Dropout(0.5))


model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 16 , activation = 'relu'))
model.add(Dense(3, activation='softmax'))
# model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=50, batch_size=256, validation_split=0.2)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print("Test Accuracy:", test_accuracy)